In [118]:
import pandas as pd
from pandas.io import sql

Pandas can be used to connect to most relational databases. In this demonstration, we will create and connect to a SQLite database. SQLite creates portable SQL databases saved in a single file. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines.

If you are looking to start using a database without the setup of `mysql` or `postgres`, SQLite is a good start.

In [119]:
import sqlite3

In [120]:
conn = sqlite3.connect('dat-test.db')
conn2 = sqlite3.connect('dat-test2.db')

Let's return to the Rossmann sales data and load that into the database.

Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arugments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

In [121]:
data = pd.read_csv('../../../lesson-15/assets/dataset/rossmann.csv', low_memory=False)
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [122]:
data.to_sql('rossmann_sales',
            con=conn,
            if_exists='replace',
            index=False)


In [141]:

sql.read_sql('SELECT Promo, SUM(Sales) FROM rossmann_sales GROUP BY Promo', con=conn)

,Promo,SUM(Sales)
0,0,2771974337
1,1,3101206286


In [142]:
sql.read_sql('SELECT SUM(Sales)/SUM(Promo) FROM rossmann_sales WHERE Open=1 AND Promo=1', con=conn)

,SUM(Sales)/SUM(Promo)
0,8228


In [144]:
sql.read_sql('SELECT SUM(Sales)/COUNT(Promo) FROM rossmann_sales WHERE Open=1 AND Promo=0', con=conn)

,SUM(Sales)/COUNT(Promo)
0,5929


In [145]:
sql.read_sql('SELECT Store, SUM(Sales) FROM rossmann_sales WHERE Promo=1 AND Open=1 GROUP BY Store', con=conn).head()

,Store,SUM(Sales)
0,1,1855039
1,2,2222214
2,3,3099120
3,4,3733384
4,5,2139936


> #### CHECK: Load the Rossmann Store metadata in `rossmann-stores.csv` and create a table into the database from it

In [146]:
# TODO
rossmann_stores = pd.read_csv('../../assets/dataset/rossmann-stores.csv', low_memory=False)

rossmann_stores.to_sql('rossmann_stores', con=conn, if_exists='replace', index=False)
rossmann_stores.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales
0,1,c,a,1270,9,2008,0,NaN,NaN,NaN,1
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct",2
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct",3
3,4,c,c,620,9,2009,0,NaN,NaN,NaN,4
4,5,a,a,29910,4,2015,0,NaN,NaN,NaN,5


> #### CHECK: (SELECT) Have the students write a query that returns the `Store`, `Date` and `Customers`

In [147]:
# TODO
sql.read_sql('SELECT Store, Sales, StoreType, Assortment FROM rossmann_stores', con=conn)

,Store,Sales,StoreType,Assortment
0,1,1,c,a
1,2,2,a,a
2,3,3,a,a
3,4,4,c,c
4,5,5,a,a
5,6,6,a,a
6,7,7,a,c
7,8,8,a,a
8,9,9,a,c
9,10,10,a,a


> #### CHECK: (WHERE) Have the students write a query that returns the `Store`, `Date` and `Customers` for when the stores were open and running a promotion

In [148]:
# TODO

> #### CHECK: (GROUP BY) Have the students write a query that returns the total sales on promotion days.

In [149]:
# TODO

#### Exercises

1. Load the Walmart sales and store features data
1. Create a table for each of those datasets
1. Select the store, date and fuel price on days it was over 90 degrees
1. Select the store, date and weekly sales and temperature
1. What were average sales on holiday vs. non-holiday sales
1. What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees

In [154]:
sql.read_sql('SELECT a.Sales, s.Sales FROM rossmann_sales as a JOIN rossmann_stores as s ON a.Store = s.Store', con=conn)

,Sales,Sales
0,5263,1
1,6064,2
2,8314,3
3,13995,4
4,4822,5
5,5651,6
6,15344,7
7,8492,8
8,8565,9
9,7185,10
